## Update your GroundX client

Current version is 2.9.92.

In [62]:
pip install -U "groundx[extract]" && pip install ipywidgets


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [63]:
pip show groundx

Name: groundx
Version: 2.9.92
Summary: 
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /Users/benjaminfletcher/.pyenv/versions/3.11.0/lib/python3.11/site-packages
Requires: aiohttp, httpx, pydantic, pydantic-core, requests, tqdm, types-tqdm, typing_extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Initialize client

Update with your API key and GroundX on Prem API URL

**remember to add a `/api` to the end of your URL like `https://url.com/api`**

If you are working with an existing bucket and workflow, you must set the bucket_id and workflow_id.

In [74]:
import typing

### set this to the local yaml file and path (leave off the .yaml from the file name though)

cache_path = "./prompts"
file_name = "simple"

###

### set these if working with an existing bucket and/or workflow_id

bucket_id: typing.Optional[int] = 2
workflow_id: typing.Optional[str] = "f40981db-50a3-4075-9ea5-86fc515f09ef"

###

from groundx import GroundX

gx_client = GroundX(
    # from values/values.groundx.secret.yaml, "GROUNDX_ADMIN_API_KEY" 
    api_key="00000000-0000-0000-0000-000000000001",
    # route for groundx pod, can be found by running "oc get routes groundx"
    # don't forget /api
    base_url="https://groundx-eyelevel.apps.23629ac0-eyelevelai.openshiftpartnerlabs.com/api",
)

## Create a Bucket (optional)

This is an optional step. If you do not already have a test bucket to work with.

In [10]:
res = gx_client.buckets.create(
        name="workflow-test",
    )

if res.bucket:
    print(f"bucket_id=[{res.bucket.bucket_id}]")

    bucket_id = res.bucket.bucket_id
else:
    print(res)

bucket_id=[2]


## List Existing Buckets (optional)

An optional step to see the existing buckets.

In [36]:
res = gx_client.buckets.list()
if res.buckets:
    for b in res.buckets:
        print(f"[{b.bucket_id}]\t\t[{b.name}]")

[2]		[workflow-test]
[1]		[System Status - Search]


## Initialize the Prompt Manager

You may want to update the folder path to where your yaml files are stored.

In [65]:
from groundx.extract import Logger, Source
from manager import ExtractPromptManager

if not workflow_id:
    workflow_id = file_name

logger = Logger(name="manage-workflows", level="info")
prompt_manager = ExtractPromptManager(
    cache_source=Source(
        logger=logger,
        cache_path=cache_path,
    ),
    config_source=Source(
        logger=logger,
        cache_path=cache_path,
    ),
    logger=logger,
    default_file_name=file_name,
    default_workflow_id=workflow_id,
    gx_client=gx_client,
)

[af40bb72-ab75-48fb-b903-8d838268b036] [simple.yaml] loading cache_workflow
2025-12-18 21:24:35,656 - [45221] - INFO - 

	>> _config_source.fetch exception for [af40bb72-ab75-48fb-b903-8d838268b036] [failed to get prompt yaml [./prompts/af40bb72-ab75-48fb-b903-8d838268b036.yaml]]
trying _cache_source [simple.yaml]...



In [5]:
res = gx_client.workflows.create(
    chunk_strategy="element",
    name="latest workflow",
    # loads extract prompt from `{cache_path}/{file_name}.yaml`
    steps=prompt_manager.workflow_steps(file_name=file_name),
    # configures workflow to be an `extract` workflow
    extract=prompt_manager.workflow_extract_dict(file_name=file_name),
)

workflow_id = res.workflow.workflow_id

print(f"[{res.workflow.workflow_id}]\t\t[{res.workflow.name}]")

[c395c1cb-7c88-49d1-a44a-509ed0478372]		[latest workflow]


## List Existing Workflows (optional)

An optional step to see the existing workflows.

In [77]:
res = gx_client.workflows.list()

for w in res.workflows:
    print(f"[{w.workflow_id}]\t\t[{w.name}]")
    if w.relationships:
        print(f"\tis account default [{w.relationships.account}]\t\tattached to buckets [{w.relationships.ids}]")

[1a2a3c7f-01c6-45b3-8586-cc47ab2268fa]		[account default]
	is account default [False]		attached to buckets [None]
[2b1dbef5-cac8-4ba9-88a8-f3e12d85422b]		[account default]
	is account default [False]		attached to buckets [None]
[37859009-40ee-4622-81b7-815d2b0e3cb3]		[account default]
	is account default [False]		attached to buckets [None]
[6ab51ef2-7a92-4478-9006-5df2b56e33fd]		[account default]
	is account default [False]		attached to buckets [None]
[9bc415a6-051e-429a-b3a5-fda50e1229b3]		[account default]
	is account default [False]		attached to buckets [None]
[af40bb72-ab75-48fb-b903-8d838268b036]		[account default]
	is account default [False]		attached to buckets [None]
[b45db8d7-73b5-4837-8e6d-09356f03f14e]		[account default]
	is account default [False]		attached to buckets [None]
[c395c1cb-7c88-49d1-a44a-509ed0478372]		[latest workflow]
	is account default [False]		attached to buckets [None]
[ddef9973-9fcb-463a-a014-ed5eb66f4871]		[account default]
	is account default [False]		a

## Get Workflow (optional)

An optional step to get workflow by account, bucket ID, or workflow ID.

Set bucket_id and workflow_id if not already set.

In [76]:
from groundx.core import ApiError

try:
    res = gx_client.workflows.get_account()

    print(f"get workflow for account\n\t[{res.workflow.workflow_id}]\t\t[{res.workflow.name}]\n")
except ApiError as e:
    print(f"get workflow for account\n\t[{e.status_code}]\t{e.body}\n")

if not bucket_id:
    raise Exception(f"set bucket_id in the Initialize client step")

try:
    res = gx_client.workflows.get(id=bucket_id)

    print(f"get workflow by bucket_id [{bucket_id}]\n\t[{res.workflow.workflow_id}]\t\t[{res.workflow.name}]\n")
except ApiError as e:
    print(f"get workflow by bucket_id [{bucket_id}]\n\t[{e.status_code}]\t{e.body}\n")

if not workflow_id:
    raise Exception(f"set workflow_id in the Initialize client step")

try:
    res = gx_client.workflows.get(workflow_id)

    print(f"get workflow by workflow_id [{workflow_id}]\n\t[{res.workflow.workflow_id}]\t\t[{res.workflow.name}]")
    if res.workflow.relationships:
        print(f"\t\tis account default [{res.workflow.relationships.account}]\t\tattached to buckets [{res.workflow.relationships.ids}]")
except ApiError as e:
    print(f"get workflow by workflow_id [{workflow_id}]\n\t[{e.status_code}]\t{e.body}")

get workflow for account
	[f40981db-50a3-4075-9ea5-86fc515f09ef]		[account default]

get workflow by bucket_id [2]
	[400]	{'message': 'You do not have access to a workflow with this ID'}

get workflow by workflow_id [f40981db-50a3-4075-9ea5-86fc515f09ef]
	[f40981db-50a3-4075-9ea5-86fc515f09ef]		[account default]
		is account default [True]		attached to buckets [None]


## Update Workflow

Update an existing workflow. You must set the workflow_id to the GroundX ID.

In [75]:
if not workflow_id or not file_name:
    raise Exception(f"set file_name and workflow_id in the Initialize client step")

res = prompt_manager.update_prompts(
    file_name=file_name,
    workflow_id=workflow_id,
)

print(f"[{res.workflow.workflow_id}] [{res.workflow.name}]")

2025-12-18 21:54:57,505 - [45221] - INFO - 

	>> _config_source.fetch exception for [f40981db-50a3-4075-9ea5-86fc515f09ef] [failed to get prompt yaml [./prompts/f40981db-50a3-4075-9ea5-86fc515f09ef.yaml]]
trying _cache_source [simple.yaml]...

[f40981db-50a3-4075-9ea5-86fc515f09ef] [account default]


## Assign to Account as the Default Prompt (optional)

An optional step to change the account default prompt.

**note: this will replace the current default account prompt**

In [7]:
if not workflow_id:
    raise Exception(f"set workflow_id in the Initialize client step")

res = gx_client.workflows.add_to_account(workflow_id=workflow_id)

print(res)

message='OK'


## Assign to a Bucket (optional)

An optional step to change the prompt assigned to a bucket.

**note: this will replace the current prompt assigned to the bucket**

In [13]:
if not bucket_id:
    bucket_id = 0
if not workflow_id:
    workflow_id = ""

res = gx_client.workflows.add_to_id(id=bucket_id, workflow_id=workflow_id)

print(res)

message='OK'


## Removing from Bucket (optional)

An optional step to remove the prompt assigned to a bucket.

**note: this will remove the current prompt assigned to the bucket**

In [17]:
if not bucket_id:
    bucket_id = 0

res = gx_client.workflows.remove_from_id(id=bucket_id)

print(res)

message='OK'


## Removing from Account (optional)

An optional step to remove the prompt assigned to the account.

**note: this will remove the current prompt assigned to the account**

In [11]:
res = gx_client.workflows.remove_from_account()

print(res)

message='OK'


## Extract Information from a File

Upload an invoice for information extraction.

In [78]:
from groundx import Document

res = gx_client.ingest(
    documents=[
        Document(
            bucket_id=bucket_id,
            file_path="./test-docs/t-mobile.pdf",
        ),
    ],
)

process_id = res.ingest.process_id
print(f"process_id = [{process_id}]")

process_id = [4c5616a5-33ae-441e-b100-0891ac281954]


## Query the Status of a File

Check the processing status of a file.

In [97]:
res = gx_client.documents.get_processing_status_by_id(
    process_id=process_id,
)

print(res.ingest.model_dump_json(indent=2))

{
  "id": null,
  "process_id": "4c5616a5-33ae-441e-b100-0891ac281954",
  "progress": {
    "cancelled": null,
    "complete": {
      "documents": [
        {
          "bucket_id": 2,
          "document_id": "8291ce45-1ee7-4f51-a4dc-920d2a29b723",
          "file_name": "t-mobile.pdf",
          "file_size": "3 KB",
          "file_type": "pdf",
          "filter": null,
          "process_id": "4c5616a5-33ae-441e-b100-0891ac281954",
          "process_level": null,
          "search_data": null,
          "source_url": "https://upload.eyelevel.ai/prod/file/ssp/fdfaa083-de3f-48d9-830c-554aa107c4c9.pdf",
          "status": "complete",
          "status_message": null,
          "text_url": "http://minio-eyelevel.apps.23629ac0-eyelevelai.openshiftpartnerlabs.com/eyelevel/layout/processed/4c5616a5-33ae-441e-b100-0891ac281954/8291ce45-1ee7-4f51-a4dc-920d2a29b723-extract.json",
          "xray_url": "http://minio-eyelevel.apps.23629ac0-eyelevelai.openshiftpartnerlabs.com/eyelevel/layout

## Downloading Extractions

The extractions are in the file `text_url` when the document is completed.

In [100]:
import requests

res = requests.get("http://minio-eyelevel.apps.23629ac0-eyelevelai.openshiftpartnerlabs.com/eyelevel/layout/processed/4c5616a5-33ae-441e-b100-0891ac281954/8291ce45-1ee7-4f51-a4dc-920d2a29b723-extract.json")

res.json()

{'due_date': '2024-02-28',
 'amount_due': 110.7,
 'provider_name': 'T-Mobile',
 'account_number': '100469352',
 'service_address': 'FIRST NAME LAST NAME ADDRESS CITY, STATE, ZIP CODE'}